# [Fannie Mae Single-Family Loan Performance Data](https://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html)

## About the Dataset

| Data Subset | Size of Compressed Dataset (GB) | Size of Uncompressed Dataset (GB) | Size of Acquistion Dataset (GB) | Size of Performance Dataset (GB) | Number of Loans (Millions) | Number of Performance Records (Billions) |
| ------- | ----------------------------- | ---------------------------- | -------- | ----------| ---------------- | ---------------------------------------- |
| mortgage_2000      | 0.45 | 3.9 | 0.137 | 3.7 | 1.21 | 0.036 |
| mortgage_2000-2001 | 1.9 | 16 | 0.473 | 16 | 4.22 | 0.148 |
| mortgage_2000-2003 | 9.3 | 78 | 1.5 | 76 | 13.5 | 0.743 |
| mortgage_2000-2007 | 15 | 117 | 2.1 | 115 | 19.1 | 1.12 |
| mortgage_2000-2015 | 23 | 192 | 3.8 | 188 | 34.7 | 1.85 |
| mortgage_2000-2016 | 24 | 196 | 4.1 | 192 | 37.0 | 1.89 |

## Import Python Modules

In [ ]:
# Import all of the python modules from The Python Standard Library
# required to run the notebook.

import os
import glob
import collections

In [ ]:
# Import all standard data science and machine learning python modules 
# required to run the notebook.

import dask
import dask.dataframe
import dask.distributed
#import matplotlib
import numpy
import pandas
import sklearn

In [ ]:
# Import NVIDIA's new RAPIDS python modules designed to perform 
# GPU-acclerated data science and machine learning tasks.

import cudf
import cuml

## Configure Notebook

In [ ]:
# Define system environment variables

USER = os.environ['USER']

In [ ]:
# Define a path to the top-level directory of a local or network
# filesystem that is available and accessible to you on the system 
# where you are running the notebook.

DATA_ROOT_DIR = os.path.join('/oasis/scratch/comet', USER, 'temp_project')

In [ ]:
# Choose which version of the Fannie Mae Single-Family Loan 
# Performance (SFLP) dataset you wish to analyze.

SFLP_DATASET = 'mortgage_2000-2016'

In [ ]:
# Define a path to the directory of the filesystem where you want to 
# download the compressed SFLP dataset, extract it into its uncompressed
# form, and store for your analysis.

SFLP_DATASET_DIR = os.path.join(DATA_ROOT_DIR, 
                                'data/fannie-mae/single-family-loan-performance',
                                SFLP_DATASET)

In [ ]:
# Define the two paths to the subdirectories where the loan acquisition
# and performance datasets will be and/or are stored.

SFLP_ACQ_DATA_DIR = os.path.join(SFLP_DATASET_DIR, 'acq')
SFLP_PERF_DATA_DIR = os.path.join(SFLP_DATASET_DIR, 'perf')

## Download and Extract Dataset

In [ ]:
# Download the compressed SFLP dataset and extract it into its 
# uncompressed form, if it's not already available on the system where
# you're running the notebook.

SFLP_DATA_ROOT_URL = 'http://rapidsai-data.s3-website.us-east-2.amazonaws.com/notebook-mortgage-data'
if not os.path.exists(SFLP_DATASET_DIR):
    os.makedirs(SFLP_DATASET_DIR)
    os.chdir(SFLP_DATASET_DIR)
    os.system('wget ' + SFLP_DATA_ROOT_URL + '/' + SFLP_DATASET + '.tgz')
    os.system('tar -xf ' + SFLP_DATASET + '.tgz')
    print('SFLP dataset downloaded and extracted.')
else:
    print('SFLP dataset directory already exists. Check if dataset is already available.')

In [ ]:
# Check the contents of SFLP dataset directory.

print(os.listdir(SFLP_DATASET_DIR))

## Dataset Preparation and Loading

### Set Analysis Interval

In [ ]:
# Select the first year and quarter to begin your analysis as well as
# the last year and quarter to end your analysis.

sflp_begin_year = 2000
sflp_begin_quarter = 1

sflp_end_year = 2016
sflp_end_quarter = 4

In [ ]:
# Create two lists of all the quarterly loan acquisition and performance
# data files from the SFLP dataset that are contained within the time 
# interval defined above.

sflp_acq_files = list()
sflp_perf_files = list()

for sflp_year in list(range(sflp_begin_year, sflp_end_year+1)):
    
    # Determine the number of quarters in each year to analyze
    if (sflp_year == sflp_begin_year):
        sflp_quarters = list(range(sflp_begin_quarter, 5))
    elif (sflp_year == sflp_end_year):
        sflp_quarters = list(range(1, sflp_end_quarter+1))
    else:
        sflp_quarters = list(range(1, 5))
        
    # Get absolute path for each quarterly loan acquisition and 
    # performance data file
    for sflp_quarter in sflp_quarters:
        sflp_acq_files.extend(
            glob.glob(
                os.path.abspath(
                    os.path.join(SFLP_ACQ_DATA_DIR, 'Acquisition_' + 
                                 str(sflp_year) + 'Q' + 
                                 str(sflp_quarter) + '.*'))))
        sflp_perf_files.extend(
            glob.glob(
                os.path.abspath(
                    os.path.join(SFLP_PERF_DATA_DIR, 'Performance_' + 
                                 str(sflp_year) + 'Q' + 
                                 str(sflp_quarter) + '.*'))))

### Acquisition Data

In [ ]:
# List all of the files from the loan acquisition dataset to be analyzed
# as well as the size of each file in bytes.

for sflp_acq_file in sflp_acq_files:
    print(os.path.basename(sflp_acq_file), os.stat(sflp_acq_file).st_size)

In [ ]:
# Print the first few lines from the first loan acquisition data file.

number_of_lines = 5
with open(sflp_acq_files[0], 'r') as sflp_acq_file:
    for x in range(number_of_lines):
        print(sflp_acq_file.readline())

In [ ]:
# Define an ordered dictionary of column datatypes for the SFLP loan 
# acquisition dataset.

sflp_acq_columns = collections.OrderedDict()
sflp_acq_columns['loan_identifier'] = 'Int64'
sflp_acq_columns['origination_channel'] = 'category'
sflp_acq_columns['seller_name'] = 'category'
sflp_acq_columns['original_interest_rate'] = 'float64'
sflp_acq_columns['original_unpaid_principal_balance'] = 'Int64'
sflp_acq_columns['original_loan_term'] = 'Int64'
sflp_acq_columns['origination_date'] = 'object'
sflp_acq_columns['first_payment_date'] ='object'
sflp_acq_columns['original_loan_to_value'] = 'float64'
sflp_acq_columns['original_combined_loan_to_value'] = 'float64'
sflp_acq_columns['number_of_borrowers'] = 'Int64'
sflp_acq_columns['original_debt_to_income_ratio'] = 'float64'
sflp_acq_columns['borrower_credit_score_at_origination'] = 'Int64'
sflp_acq_columns['first_time_home_buyer_indicator'] = 'category'
sflp_acq_columns['loan_purpose'] = 'category'
sflp_acq_columns['property_type'] = 'category'
sflp_acq_columns['number_of_units'] = 'Int64'
sflp_acq_columns['occupancy_type'] = 'category'
sflp_acq_columns['property_state'] = 'category'
sflp_acq_columns['zip_code_short'] = 'Int64'
sflp_acq_columns['primary_mortgage_insurance_percent'] = 'float64'
sflp_acq_columns['product_type'] = 'category'
sflp_acq_columns['co-borrower_credit_score_at_origination'] = 'Int64'
sflp_acq_columns['mortgage_insurance_type'] = 'Int64'
sflp_acq_columns['relocation_mortgage_indicator'] = 'category'
sflp_acq_columns['year_quarter'] = 'Int64'

In [ ]:
%%time
# Dask
# 
# Read in all of the loan acquisition data into a Pandas DataFrame 
# (sflp_acq_pdf) via the distributed dask.dataframe.read_csv method.

dask_client = dask.distributed.Client(n_workers=32, threads_per_worker=2)
sflp_acq_ddf = dask.dataframe.read_csv(sflp_acq_files, 
                                       names=[*sflp_acq_columns.keys()], 
                                       delimiter='|', 
                                       dtype=sflp_acq_columns)
sflp_acq_ddf = dask_client.persist(sflp_acq_ddf)
sflp_acq_pdf = sflp_acq_ddf.compute()
dask_client.close()

In [ ]:
# Pandas
#
# Check datatypes of the loan acquisition Pandas DataFrame (sflp_acq_pdf).

sflp_acq_pdf.dtypes

In [ ]:
# Pandas
#
# Convert the date-related object datatypes in the loan acquisition
# Pandas DataFrame (sflp_acq_pdf) to datetime datatypes.

sflp_acq_pdf['origination_date'] = pandas.to_datetime(
    sflp_acq_pdf['origination_date'], format='%m/%Y')
sflp_acq_pdf['first_payment_date'] = pandas.to_datetime(
    sflp_acq_pdf['first_payment_date'], format='%m/%Y')

In [ ]:
# Pandas
#
# Re-check the datatypes of the loan acquisition Pandas DataFrame
# (sflp_acq_pdf) after completion of the object-to-datetime conversion. 

sflp_acq_pdf.dtypes

In [ ]:
%%time 
# cuDF
#
# Create a copy of the loan acquisition Pandas DataFrame (sflp_acq_pdf)
# in the memory of a single NVIDIA GPU by creating a cuDF DataFrame
# (sflp_acq_cudf) from the Pandas DataFrame.

sflp_acq_cudf = cudf.DataFrame.from_pandas(sflp_acq_pdf)

In [ ]:
# cuDF
#
# Check that the datatypes of the loan acquisition cuDF DataFrame 
# (sflp_acq_cudf) are in fact the same datatypes as those of the Pandas
# DataFrame (sflp_acq_pdf).

sflp_acq_cudf.dtypes

In [ ]:
# Pandas
#
# Inspect the first few rows of the loan acquisition Pandas DataFrame
# (sflp_acq_pdf).

sflp_acq_pdf.head()

In [ ]:
# cuDF
#
# Compare the first few rows of the loan acquisition cuDF DataFrame
# (sflp_acq_cudf) against those of the Pandas DataFrame (sflp_acq_pdf).

sflp_acq_cudf.head()

### Performance Data

## Data Exploration and Manipulation

### Filtering and Querying Data

In [ ]:
%%time
# Pandas
#
# How many loans were made for more than $400,000?

len(sflp_acq_pdf.query('original_unpaid_principal_balance > 400000'))

In [ ]:
%%time
# cuDF
#
# How many loans were made for more than $400,000?

len(sflp_acq_cudf.query('original_unpaid_principal_balance > 400000'))

In [ ]:
%%time
# Pandas
#
# How many loans were made in California for more than $400,000?

len(sflp_acq_pdf.query('property_state == "CA" and \
                        original_unpaid_principal_balance > 400000'))

In [ ]:
%%time
# cuDF (FAIL)
#
# How many loans were made in California for more than $400,000?

len(sflp_acq_cudf.query('property_state == "CA" and \
                        original_unpaid_principal_balance > 400000'))

In [ ]:
# cuDF
#
# Check the list of categories in the property_state column. Find CA.

sflp_acq_cudf.property_state.cat.categories

In [ ]:
%%time 
# cuDF
#
# It appears that cuDF still does not support literal string comparisons
# in the cudf.DataFrame.query() method. e.g., instead, you must use the
# numerical index one of the categories in a category datatype. 
#
# How many loans were made in California (4) for more than $400,000?

len(sflp_acq_cudf.query('property_state == 4 and \
                         original_unpaid_principal_balance > 400000'))

In [ ]:
%%time 
# Pandas
# 
# How many loans were made in either California, New York, or Florida
# for more than $400,000 to principal borrowers who had a credit score
# of less than 650?

len(sflp_acq_pdf.query('(property_state == "CA" or \
                         property_state == "NY" or \
                         property_state =="FL") and \
                         original_unpaid_principal_balance > 400000 and \
                         borrower_credit_score_at_origination < 650'))

In [ ]:
%%time
# cuDF
# 
# How many loans were made in either California (4), New York (34), or 
# Florida (9) for more than $400,000 to principal borrowers who had a 
# credit score of less than 650?

len(sflp_acq_cudf.query('(property_state == 4 or \
                          property_state == 34 or \
                          property_state == 9) and \
                          original_unpaid_principal_balance > 400000 and \
                          borrower_credit_score_at_origination < 650'))

### Grouping Data

In [ ]:
%%time
# Pandas   .count(Int64)
#
# What was the total number of loans originated per month? 

total_number_of_loans_originated_per_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['loan_identifier'].count().to_frame()

In [ ]:
%%time
# cuDF    .count(int64)
#
# What was the total number of loans originated per month?
# .count(Int64)

total_number_of_loans_originated_per_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['loan_identifier'].count().to_frame()

In [ ]:
%%time
# Pandas   .sum(Int64)
# 
# What was the total unpaid principal balance originated per month (in
# billions of dollars)?

total_unpaid_principal_balance_originated_per_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame().div(1000000000)

In [ ]:
%%time
# cuDF    .sum(int64)   (FAIL)
# 
# What was the total unpaid principal balance originated per month (in
# billions of dollars)?

total_unpaid_principal_balances_originated_per_month_cudf  = sflp_acq_cudf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame().div(1000000000)

In [ ]:
%%time
# Pandas   .sum(Int64)
# 
# What was the total unpaid principal balance originated per month?

total_unpaid_principal_balance_originated_per_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame()

In [ ]:
%%time
# cuDF    .sum(int64)
# 
# What was the total unpaid principal balance originated per month?

total_unpaid_principal_balance_originated_per_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['original_unpaid_principal_balance'].sum().to_frame()

In [ ]:
%%time
# Pandas   .mean(Int64)
# 
# What was the average unpaid principal balance at originiation by month?

average_unpaid_principal_balance_at_origination_by_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_unpaid_principal_balance'].mean().to_frame()

In [ ]:
%%time
# cuDF    .mean(int64)    
# 
# What was the average unpaid principal balance at originiation by month?

average_unpaid_principal_balance_at_origination_by_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['original_unpaid_principal_balance'].mean().to_frame()

In [ ]:
%%time
# Pandas   .mean(float64)
#
# What was the average loan to value ratio at originiation by month?

average_loan_to_value_at_origination_by_month_pdf  = sflp_acq_pdf.groupby(
    'origination_date')['original_loan_to_value'].mean().to_frame()

In [ ]:
%%time
# cuDF    .mean(float64)
#
# What was the average loan to value ratio at originiation by month?

average_loan_to_value_at_origination_by_month_cudf  = sflp_acq_cudf.groupby(
    'origination_date')['original_loan_to_value'].mean().to_frame()

In [ ]:
%time
# Pandas
#
# What was the average credit score of a borrower at originiation?

average_credit_score_at_origination_by_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['borrower_credit_score_at_origination'].mean().to_frame()

In [ ]:
%time
# cuDF
#
# What was the average credit score of a borrower at originiation?

average_credit_score_at_origination_by_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['borrower_credit_score_at_origination'].mean().to_frame()

In [ ]:
%%time
# Pandas
#
# What was the average debt-to-income ratio of a borrower at origination?

average_debt_to_income_at_origination_by_month_pdf  = sflp_acq_pdf.groupby(
    'origination_date')['original_debt_to_income_ratio'].mean().to_frame()

In [ ]:
%%time
# cuDF
#
# What was the average debt-to-income ratio of a borrower at origination?

average_debt_to_income_at_origination_by_month_cudf  = sflp_acq_cudf.groupby(
    'origination_date')['original_debt_to_income_ratio'].mean().to_frame()

In [ ]:
%%time
# Pandas
# 
# What was the average interest rate on a loan at origination?

average_interest_rate_at_origination_by_month_pdf = sflp_acq_pdf.groupby(
    'origination_date')['original_interest_rate'].mean().to_frame()

In [ ]:
%%time
# cuDF
# 
# What was the average interest rate on a loan at origination?

average_interest_rate_at_origination_by_month_cudf = sflp_acq_cudf.groupby(
    'origination_date')['original_interest_rate'].mean().to_frame()

### Concatenating Data

In [ ]:
%%time
# Pandas
#
# Combine the monthly summary statistics series created above into a
# single pandas DataFrame.

sflp_acq_monthly_summary_pdf = pandas.concat([total_number_of_loans_originated_per_month_pdf, 
                                      total_unpaid_principal_balance_originated_per_month_pdf, 
                                      average_unpaid_principal_balance_at_origination_by_month_pdf, 
                                      average_loan_to_value_at_origination_by_month_pdf, 
                                      average_credit_score_at_origination_by_month_pdf, 
                                      average_debt_to_income_at_origination_by_month_pdf, 
                                      average_interest_rate_at_origination_by_month_pdf], axis=1)

In [ ]:
%%time
# cuDF
#
# Combine the monthly summary statistics series created above into a
# single pandas DataFrame.

sflp_acq_monthly_summary_cudf = cudf.concat([total_number_of_loans_originated_per_month_cudf, 
                                      total_unpaid_principal_balance_originated_per_month_cudf, 
                                      average_unpaid_principal_balance_at_origination_by_month_cudf, 
                                      average_loan_to_value_at_origination_by_month_cudf, 
                                      average_credit_score_at_origination_by_month_cudf, 
                                      average_debt_to_income_at_origination_by_month_cudf, 
                                      average_interest_rate_at_origination_by_month_cudf], axis=1)

## Cluster Analysis and Classification